In [1]:
#write your names and students ids here

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from gensim.models import Word2Vec, KeyedVectors


from scipy.sparse import dok_matrix
# Install spaCy (run in terminal/prompt)
import sys
#!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
#!{sys.executable} -m spacy download en



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\UX550VD\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\UX550VD\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
from sklearn import preprocessing

In [4]:
data=pd.read_json ('../News_Category_Dataset_v2.json', lines=True)

### Typical data analysis of a dataframe now

In [5]:
data.shape


(200853, 6)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [7]:
#data.describe
data.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [8]:
print(data.groupby('category').size())

category
ARTS               1509
ARTS & CULTURE     1339
BLACK VOICES       4528
BUSINESS           5937
COLLEGE            1144
COMEDY             5175
CRIME              3405
CULTURE & ARTS     1030
DIVORCE            3426
EDUCATION          1004
ENTERTAINMENT     16058
ENVIRONMENT        1323
FIFTY              1401
FOOD & DRINK       6226
GOOD NEWS          1398
GREEN              2622
HEALTHY LIVING     6694
HOME & LIVING      4195
IMPACT             3459
LATINO VOICES      1129
MEDIA              2815
MONEY              1707
PARENTING          8677
PARENTS            3955
POLITICS          32739
QUEER VOICES       6314
RELIGION           2556
SCIENCE            2178
SPORTS             4884
STYLE              2254
STYLE & BEAUTY     9649
TASTE              2096
TECH               2082
THE WORLDPOST      3664
TRAVEL             9887
WEDDINGS           3651
WEIRD NEWS         2670
WELLNESS          17827
WOMEN              3490
WORLD NEWS         2177
WORLDPOST          2579
dtype: 

In [9]:
#we can see that many categories are similar.
#Let´s merge them. We will merge the more specific ones into the more general ones in case of doubt
#the code in the cell below was obtained from https://www.kaggle.com/rmisra/news-category-dataset/discussion/114275

In [10]:
data['category']=data['category'].replace({"HEALTHY LIVING": "WELLNESS",
"QUEER VOICES": "GROUPS VOICES",
"BUSINESS": "BUSINESS & FINANCES",
"PARENTS": "PARENTING",
"BLACK VOICES": "GROUPS VOICES",
"THE WORLDPOST": "WORLD NEWS",
"STYLE": "STYLE & BEAUTY",
"GREEN": "ENVIRONMENT",
"TASTE": "FOOD & DRINK",
"WORLDPOST": "WORLD NEWS",
"SCIENCE": "SCIENCE & TECH",
"TECH": "SCIENCE & TECH",
"MONEY": "BUSINESS & FINANCES",
"ARTS": "ARTS & CULTURE",
"COLLEGE": "EDUCATION",
"LATINO VOICES": "GROUPS VOICES",
"CULTURE & ARTS": "ARTS & CULTURE",
"FIFTY": "MISCELLANEOUS",
"GOOD NEWS": "MISCELLANEOUS"})

In [11]:
print(data.groupby('category').size())

category
ARTS & CULTURE          3878
BUSINESS & FINANCES     7644
COMEDY                  5175
CRIME                   3405
DIVORCE                 3426
EDUCATION               2148
ENTERTAINMENT          16058
ENVIRONMENT             3945
FOOD & DRINK            8322
GROUPS VOICES          11971
HOME & LIVING           4195
IMPACT                  3459
MEDIA                   2815
MISCELLANEOUS           2799
PARENTING              12632
POLITICS               32739
RELIGION                2556
SCIENCE & TECH          4260
SPORTS                  4884
STYLE & BEAUTY         11903
TRAVEL                  9887
WEDDINGS                3651
WEIRD NEWS              2670
WELLNESS               24521
WOMEN                   3490
WORLD NEWS              8420
dtype: int64


In [12]:
(data.groupby('category').size()).std()

7299.533828002569

As we may observe, the data is still highly imbalanced. Therefore further techniques will have to be applied to
make up for this

## Data cleaning:

We must  apply either lemmatization or stemming to the words.
We will try to go for lemmatization first as it depends on the context of the word.

We used the following link to guide ourselves https://www.machinelearningplus.com/nlp/lemmatization-examples-python/


//TODO: preproces words to remove apostrophes for example. isn´t -> is not

In [13]:
lemmatizer = WordNetLemmatizer()

def clean(text):

    # Define the sentence to be lemmatized
    text=text.lower() #making everything lowercase
    
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(text)
    #print(word_list)

    # Lemmatize list of words and join
    #lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    #print(lemmatized_output)
    
   # return lemmatized_output
    return word_list



In [14]:

data['short_description'] = data['short_description'].apply(clean)
data['headline'] = data['headline'].apply(clean)


### ENCODING CLASSES

We need to transform strings into numerical labels. Remember, if we extend our model to work with NN, 
then we should transform to one-hot encoding instead

In [15]:
le = preprocessing.LabelEncoder()
le.fit(data['category'])
print(list(le.classes_))
print(le.transform(['FOOD & DRINK', 'GROUPS VOICES', 'HOME & LIVING']))


['ARTS & CULTURE', 'BUSINESS & FINANCES', 'COMEDY', 'CRIME', 'DIVORCE', 'EDUCATION', 'ENTERTAINMENT', 'ENVIRONMENT', 'FOOD & DRINK', 'GROUPS VOICES', 'HOME & LIVING', 'IMPACT', 'MEDIA', 'MISCELLANEOUS', 'PARENTING', 'POLITICS', 'RELIGION', 'SCIENCE & TECH', 'SPORTS', 'STYLE & BEAUTY', 'TRAVEL', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS', 'WOMEN', 'WORLD NEWS']
[ 8  9 10]


In [16]:
data['category encoded'] = le.transform(data['category'])
data.head()

,category,headline,authors,link,short_description,date,category encoded
0,CRIME,"[there, were, 2, mass, shootings, in, texas, l...",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,"[she, left, her, husband, ., he, killed, their...",2018-05-26,3
1,ENTERTAINMENT,"[will, smith, joins, diplo, and, nicky, jam, f...",Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,"[of, course, it, has, a, song, .]",2018-05-26,6
2,ENTERTAINMENT,"[hugh, grant, marries, for, the, first, time, ...",Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,"[the, actor, and, his, longtime, girlfriend, a...",2018-05-26,6
3,ENTERTAINMENT,"[jim, carrey, blasts, 'castrato, ', adam, schi...",Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,"[the, actor, gives, dems, an, ass-kicking, for...",2018-05-26,6
4,ENTERTAINMENT,"[julianna, margulies, uses, donald, trump, poo...",Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"[the, ``, dietland, '', actress, said, using, ...",2018-05-26,6


### Splitting into train and test



In [26]:
train=data.sample(frac=0.8,random_state=200) #random state is a seed value
test=data.drop(train.index)

train.index= range(len(train))
test.reset_index= range(len(test))


In [55]:
test.reset_index

range(0, 40171)

### OUR MODELS

In [1]:
#the code from assignment 2 was changed here. We had a framework to start with this way.

class FeatureCreator:
    def __init__(self):
        self.voc = {}
        self.id = {}
        self.count = 0
    
    
    def createFeatureSet(self, data):
        # t1 and t2 are the 2 questions we want to compare
        for index, new in data.iterrows():
            
            for w in new['headline']:
                
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1;
                    
            for w in new['short_description']:
                if w in self.voc:
                    self.voc[w] += 1
                else:
                    self.voc[w] = 1
                    self.id[w] = self.count
                    self.count += 1;
                
        

class BowFeatureCreator(FeatureCreator): #runs in O(n^2)

    
     def createFeatures(self,data):
        
        id_len=len(self.id) #we will add this length to the index if we want to access the second question. It is made clear in the implementation

        features = dok_matrix((len(data),self.count*2)) #we will have twice the entries. One |VOC| x |2| one for the headline and the other one for the description
        print(features.shape)
        label = []
        i=0
        for i, row in data.iterrows():
            
            for w in row['headline']:
                if w in self.id:
                    features[i,self.id[w]] += 1
                    #features[i,self.id[w]+self.count] += 1
            for w in row['short_description']:
                if w in self.id:
                    features[i, self.id[w] + self.count]+= 1 #now there is one bow for the headline and another for the description
                    #features[i,self.id[w] + self.count] += 1
            print('i', i)
            label.append(row['category encoded'])
            
            
            #we want to make difference independent of the sentences,
        #that is, no negative values. We must take absolute value of features
        
        """
            # not the most efficient thing ever here, but it does the job
        i=0
        for j, row in data.iterrows():
            for w in row['headline']:
                if w in self.id:
                    if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w]+self.count] *= -1
            for w in row['short_description']:
                if w in self.id:
                     if features[j,self.id[w]+self.count]<0:
                        features[j,self.id[w] + self.count] *= -1
            i+=1.
            if i%1000 == 0:
                print('iteration: ', i)

        #features=self.absoluteValue(features,data)

        """
        
        
        return features,label
    



        

Calculate size of the vocabulary:

In [2]:
%%time
features=BowFeatureCreator()
features.createFeatureSet(train)


NameError: name 'train' is not defined

In [78]:
print(len(features.id))
print(len(features.voc))
print(features.count)

#print(train.shape)
print(len(train))


for k,v in features.id.items():
    if v > len(features.id):
        print(k)
        print(v)

105693
105693
105693
160682


In [79]:
f,l = features.createFeatures(train)
print(features.count)

(160682, 211386)
i 0
i 1
i 2
i 3
i 4
i 5
i 6
i 7
i 8
i 9
i 10
i 11
i 12
i 13
i 14
i 15
i 16
i 17
i 18
i 19
i 20
i 21
i 22
i 23
i 24
i 25
i 26
i 27
i 28
i 29
i 30
i 31
i 32
i 33
i 34
i 35
i 36
i 37
i 38
i 39
i 40
i 41
i 42
i 43
i 44
i 45
i 46
i 47
i 48
i 49
i 50
i 51
i 52
i 53
i 54
i 55
i 56
i 57
i 58
i 59
i 60
i 61
i 62
i 63
i 64
i 65
i 66
i 67
i 68
i 69
i 70
i 71
i 72
i 73
i 74
i 75
i 76
i 77
i 78
i 79
i 80
i 81
i 82
i 83
i 84
i 85
i 86
i 87
i 88
i 89
i 90
i 91
i 92
i 93
i 94
i 95
i 96
i 97
i 98
i 99
i 100
i 101
i 102
i 103
i 104
i 105
i 106
i 107
i 108
i 109
i 110
i 111
i 112
i 113
i 114
i 115
i 116
i 117
i 118
i 119
i 120
i 121
i 122
i 123
i 124
i 125
i 126
i 127
i 128
i 129
i 130
i 131
i 132
i 133
i 134
i 135
i 136
i 137
i 138
i 139
i 140
i 141
i 142
i 143
i 144
i 145
i 146
i 147
i 148
i 149
i 150
i 151
i 152
i 153
i 154
i 155
i 156
i 157
i 158
i 159
i 160
i 161
i 162
i 163
i 164
i 165
i 166
i 167
i 168
i 169
i 170
i 171
i 172
i 173
i 174
i 175
i 176
i 177
i 178
i 179
i 180
i 181
i

i 2274
i 2275
i 2276
i 2277
i 2278
i 2279
i 2280
i 2281
i 2282
i 2283
i 2284
i 2285
i 2286
i 2287
i 2288
i 2289
i 2290
i 2291
i 2292
i 2293
i 2294
i 2295
i 2296
i 2297
i 2298
i 2299
i 2300
i 2301
i 2302
i 2303
i 2304
i 2305
i 2306
i 2307
i 2308
i 2309
i 2310
i 2311
i 2312
i 2313
i 2314
i 2315
i 2316
i 2317
i 2318
i 2319
i 2320
i 2321
i 2322
i 2323
i 2324
i 2325
i 2326
i 2327
i 2328
i 2329
i 2330
i 2331
i 2332
i 2333
i 2334
i 2335
i 2336
i 2337
i 2338
i 2339
i 2340
i 2341
i 2342
i 2343
i 2344
i 2345
i 2346
i 2347
i 2348
i 2349
i 2350
i 2351
i 2352
i 2353
i 2354
i 2355
i 2356
i 2357
i 2358
i 2359
i 2360
i 2361
i 2362
i 2363
i 2364
i 2365
i 2366
i 2367
i 2368
i 2369
i 2370
i 2371
i 2372
i 2373
i 2374
i 2375
i 2376
i 2377
i 2378
i 2379
i 2380
i 2381
i 2382
i 2383
i 2384
i 2385
i 2386
i 2387
i 2388
i 2389
i 2390
i 2391
i 2392
i 2393
i 2394
i 2395
i 2396
i 2397
i 2398
i 2399
i 2400
i 2401
i 2402
i 2403
i 2404
i 2405
i 2406
i 2407
i 2408
i 2409
i 2410
i 2411
i 2412
i 2413
i 2414
i 2415
i 2416

i 4642
i 4643
i 4644
i 4645
i 4646
i 4647
i 4648
i 4649
i 4650
i 4651
i 4652
i 4653
i 4654
i 4655
i 4656
i 4657
i 4658
i 4659
i 4660
i 4661
i 4662
i 4663
i 4664
i 4665
i 4666
i 4667
i 4668
i 4669
i 4670
i 4671
i 4672
i 4673
i 4674
i 4675
i 4676
i 4677
i 4678
i 4679
i 4680
i 4681
i 4682
i 4683
i 4684
i 4685
i 4686
i 4687
i 4688
i 4689
i 4690
i 4691
i 4692
i 4693
i 4694
i 4695
i 4696
i 4697
i 4698
i 4699
i 4700
i 4701
i 4702
i 4703
i 4704
i 4705
i 4706
i 4707
i 4708
i 4709
i 4710
i 4711
i 4712
i 4713
i 4714
i 4715
i 4716
i 4717
i 4718
i 4719
i 4720
i 4721
i 4722
i 4723
i 4724
i 4725
i 4726
i 4727
i 4728
i 4729
i 4730
i 4731
i 4732
i 4733
i 4734
i 4735
i 4736
i 4737
i 4738
i 4739
i 4740
i 4741
i 4742
i 4743
i 4744
i 4745
i 4746
i 4747
i 4748
i 4749
i 4750
i 4751
i 4752
i 4753
i 4754
i 4755
i 4756
i 4757
i 4758
i 4759
i 4760
i 4761
i 4762
i 4763
i 4764
i 4765
i 4766
i 4767
i 4768
i 4769
i 4770
i 4771
i 4772
i 4773
i 4774
i 4775
i 4776
i 4777
i 4778
i 4779
i 4780
i 4781
i 4782
i 4783
i 4784

i 6728
i 6729
i 6730
i 6731
i 6732
i 6733
i 6734
i 6735
i 6736
i 6737
i 6738
i 6739
i 6740
i 6741
i 6742
i 6743
i 6744
i 6745
i 6746
i 6747
i 6748
i 6749
i 6750
i 6751
i 6752
i 6753
i 6754
i 6755
i 6756
i 6757
i 6758
i 6759
i 6760
i 6761
i 6762
i 6763
i 6764
i 6765
i 6766
i 6767
i 6768
i 6769
i 6770
i 6771
i 6772
i 6773
i 6774
i 6775
i 6776
i 6777
i 6778
i 6779
i 6780
i 6781
i 6782
i 6783
i 6784
i 6785
i 6786
i 6787
i 6788
i 6789
i 6790
i 6791
i 6792
i 6793
i 6794
i 6795
i 6796
i 6797
i 6798
i 6799
i 6800
i 6801
i 6802
i 6803
i 6804
i 6805
i 6806
i 6807
i 6808
i 6809
i 6810
i 6811
i 6812
i 6813
i 6814
i 6815
i 6816
i 6817
i 6818
i 6819
i 6820
i 6821
i 6822
i 6823
i 6824
i 6825
i 6826
i 6827
i 6828
i 6829
i 6830
i 6831
i 6832
i 6833
i 6834
i 6835
i 6836
i 6837
i 6838
i 6839
i 6840
i 6841
i 6842
i 6843
i 6844
i 6845
i 6846
i 6847
i 6848
i 6849
i 6850
i 6851
i 6852
i 6853
i 6854
i 6855
i 6856
i 6857
i 6858
i 6859
i 6860
i 6861
i 6862
i 6863
i 6864
i 6865
i 6866
i 6867
i 6868
i 6869
i 6870

i 8463
i 8464
i 8465
i 8466
i 8467
i 8468
i 8469
i 8470
i 8471
i 8472
i 8473
i 8474
i 8475
i 8476
i 8477
i 8478
i 8479
i 8480
i 8481
i 8482
i 8483
i 8484
i 8485
i 8486
i 8487
i 8488
i 8489
i 8490
i 8491
i 8492
i 8493
i 8494
i 8495
i 8496
i 8497
i 8498
i 8499
i 8500
i 8501
i 8502
i 8503
i 8504
i 8505
i 8506
i 8507
i 8508
i 8509
i 8510
i 8511
i 8512
i 8513
i 8514
i 8515
i 8516
i 8517
i 8518
i 8519
i 8520
i 8521
i 8522
i 8523
i 8524
i 8525
i 8526
i 8527
i 8528
i 8529
i 8530
i 8531
i 8532
i 8533
i 8534
i 8535
i 8536
i 8537
i 8538
i 8539
i 8540
i 8541
i 8542
i 8543
i 8544
i 8545
i 8546
i 8547
i 8548
i 8549
i 8550
i 8551
i 8552
i 8553
i 8554
i 8555
i 8556
i 8557
i 8558
i 8559
i 8560
i 8561
i 8562
i 8563
i 8564
i 8565
i 8566
i 8567
i 8568
i 8569
i 8570
i 8571
i 8572
i 8573
i 8574
i 8575
i 8576
i 8577
i 8578
i 8579
i 8580
i 8581
i 8582
i 8583
i 8584
i 8585
i 8586
i 8587
i 8588
i 8589
i 8590
i 8591
i 8592
i 8593
i 8594
i 8595
i 8596
i 8597
i 8598
i 8599
i 8600
i 8601
i 8602
i 8603
i 8604
i 8605

...

i 155885
i 155886
i 155887
i 155888
i 155889
i 155890
i 155891
i 155892
i 155893
i 155894
i 155895
i 155896
i 155897
i 155898
i 155899
i 155900
i 155901
i 155902
i 155903
i 155904
i 155905
i 155906
i 155907
i 155908
i 155909
i 155910
i 155911
i 155912
i 155913
i 155914
i 155915
i 155916
i 155917
i 155918
i 155919
i 155920
i 155921
i 155922
i 155923
i 155924
i 155925
i 155926
i 155927
i 155928
i 155929
i 155930
i 155931
i 155932
i 155933
i 155934
i 155935
i 155936
i 155937
i 155938
i 155939
i 155940
i 155941
i 155942
i 155943
i 155944
i 155945
i 155946
i 155947
i 155948
i 155949
i 155950
i 155951
i 155952
i 155953
i 155954
i 155955
i 155956
i 155957
i 155958
i 155959
i 155960
i 155961
i 155962
i 155963
i 155964
i 155965
i 155966
i 155967
i 155968
i 155969
i 155970
i 155971
i 155972
i 155973
i 155974
i 155975
i 155976
i 155977
i 155978
i 155979
i 155980
i 155981
i 155982
i 155983
i 155984
i 155985
i 155986
i 155987
i 155988
i 155989
i 155990
i 155991
i 155992
i 155993
i 155994
i 155995
i

i 157135
i 157136
i 157137
i 157138
i 157139
i 157140
i 157141
i 157142
i 157143
i 157144
i 157145
i 157146
i 157147
i 157148
i 157149
i 157150
i 157151
i 157152
i 157153
i 157154
i 157155
i 157156
i 157157
i 157158
i 157159
i 157160
i 157161
i 157162
i 157163
i 157164
i 157165
i 157166
i 157167
i 157168
i 157169
i 157170
i 157171
i 157172
i 157173
i 157174
i 157175
i 157176
i 157177
i 157178
i 157179
i 157180
i 157181
i 157182
i 157183
i 157184
i 157185
i 157186
i 157187
i 157188
i 157189
i 157190
i 157191
i 157192
i 157193
i 157194
i 157195
i 157196
i 157197
i 157198
i 157199
i 157200
i 157201
i 157202
i 157203
i 157204
i 157205
i 157206
i 157207
i 157208
i 157209
i 157210
i 157211
i 157212
i 157213
i 157214
i 157215
i 157216
i 157217
i 157218
i 157219
i 157220
i 157221
i 157222
i 157223
i 157224
i 157225
i 157226
i 157227
i 157228
i 157229
i 157230
i 157231
i 157232
i 157233
i 157234
i 157235
i 157236
i 157237
i 157238
i 157239
i 157240
i 157241
i 157242
i 157243
i 157244
i 157245
i

i 158385
i 158386
i 158387
i 158388
i 158389
i 158390
i 158391
i 158392
i 158393
i 158394
i 158395
i 158396
i 158397
i 158398
i 158399
i 158400
i 158401
i 158402
i 158403
i 158404
i 158405
i 158406
i 158407
i 158408
i 158409
i 158410
i 158411
i 158412
i 158413
i 158414
i 158415
i 158416
i 158417
i 158418
i 158419
i 158420
i 158421
i 158422
i 158423
i 158424
i 158425
i 158426
i 158427
i 158428
i 158429
i 158430
i 158431
i 158432
i 158433
i 158434
i 158435
i 158436
i 158437
i 158438
i 158439
i 158440
i 158441
i 158442
i 158443
i 158444
i 158445
i 158446
i 158447
i 158448
i 158449
i 158450
i 158451
i 158452
i 158453
i 158454
i 158455
i 158456
i 158457
i 158458
i 158459
i 158460
i 158461
i 158462
i 158463
i 158464
i 158465
i 158466
i 158467
i 158468
i 158469
i 158470
i 158471
i 158472
i 158473
i 158474
i 158475
i 158476
i 158477
i 158478
i 158479
i 158480
i 158481
i 158482
i 158483
i 158484
i 158485
i 158486
i 158487
i 158488
i 158489
i 158490
i 158491
i 158492
i 158493
i 158494
i 158495
i

i 159558
i 159559
i 159560
i 159561
i 159562
i 159563
i 159564
i 159565
i 159566
i 159567
i 159568
i 159569
i 159570
i 159571
i 159572
i 159573
i 159574
i 159575
i 159576
i 159577
i 159578
i 159579
i 159580
i 159581
i 159582
i 159583
i 159584
i 159585
i 159586
i 159587
i 159588
i 159589
i 159590
i 159591
i 159592
i 159593
i 159594
i 159595
i 159596
i 159597
i 159598
i 159599
i 159600
i 159601
i 159602
i 159603
i 159604
i 159605
i 159606
i 159607
i 159608
i 159609
i 159610
i 159611
i 159612
i 159613
i 159614
i 159615
i 159616
i 159617
i 159618
i 159619
i 159620
i 159621
i 159622
i 159623
i 159624
i 159625
i 159626
i 159627
i 159628
i 159629
i 159630
i 159631
i 159632
i 159633
i 159634
i 159635
i 159636
i 159637
i 159638
i 159639
i 159640
i 159641
i 159642
i 159643
i 159644
i 159645
i 159646
i 159647
i 159648
i 159649
i 159650
i 159651
i 159652
i 159653
i 159654
i 159655
i 159656
i 159657
i 159658
i 159659
i 159660
i 159661
i 159662
i 159663
i 159664
i 159665
i 159666
i 159667
i 159668
i

iteration:  1000.0
iteration:  2000.0
iteration:  3000.0
iteration:  4000.0
iteration:  5000.0
iteration:  6000.0
iteration:  7000.0
iteration:  8000.0
iteration:  9000.0
iteration:  10000.0
iteration:  11000.0
iteration:  12000.0
iteration:  13000.0
iteration:  14000.0
iteration:  15000.0
iteration:  16000.0
iteration:  17000.0
iteration:  18000.0
iteration:  19000.0
iteration:  20000.0
iteration:  21000.0
iteration:  22000.0
iteration:  23000.0
iteration:  24000.0
iteration:  25000.0
iteration:  26000.0
iteration:  27000.0
iteration:  28000.0
iteration:  29000.0
iteration:  30000.0
iteration:  31000.0
iteration:  32000.0
iteration:  33000.0
iteration:  34000.0
iteration:  35000.0
iteration:  36000.0
iteration:  37000.0
iteration:  38000.0
iteration:  39000.0
iteration:  40000.0
iteration:  41000.0
iteration:  42000.0
iteration:  43000.0
iteration:  44000.0
iteration:  45000.0
iteration:  46000.0
iteration:  47000.0
iteration:  48000.0
iteration:  49000.0
iteration:  50000.0
iteration

### Multinomial Naive Bayes Classifier

In [75]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(f, l)

ValueError: Found input variables with inconsistent numbers of samples: [160682, 0]

### MODEL EVALUATION

We need to create many more variations with many different features

In [ ]:
from sklearn.metrics import classification_report
testf,testl = features.createFeatures(test)
predicted_nb=mnb.predict(testf)
print(metrics.classification_report(testl, predicted_nb));

In [ ]:
.
.
.
.
.
..
.
.
.

.
.
.
.
..
.

.
.
.
.
.
.
.
..

.
.
.Ignore from here onwards

# IGNORE FROM HERE ONWARDS

//we will trin the model once we are done with the cleaning

First we split the data such that the proportion of category labels is the same in train and test. Remember, category will be our target

In [ ]:
train, test = train_test_split(data, test_size=0.2, stratify=data['category'])

Leave that for later, do not pay too much attention for now

In [ ]:

class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)))

Then you use the class weights during the training process:

train_history = model.fit( train_dataset, steps_per_epoch=n_steps, class_weight=class_weights )

In [ ]:
data.loc[1,'headline']

In [ ]:
sentences=[]
for i, row in data.iterrows():
    sentences.append(row['headline'])

model_embeddings=Word2Vec(vector_size=20, min_count=1)
model_embeddings.build_vocab(sentences, progress_per=10000)
model_embeddings.train(sentences, total_examples=model_embeddings.corpus_count, epochs=30,report_delay=1)
#model_embeddings.build(data['headlines'])


In [ ]:
model_embeddings.wv.most_similar(positive=["korea"])
model_embeddings.wv.most_similar(positive=["Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song"])